In [ ]:
!pip install imutils

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image as img_
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import matplotlib.pyplot as plt
import cv2
import seaborn as sns
import numpy as np
import os

# Hanya untuk Jupyter Notebook
# %matplotlib inline


In [ ]:
import os
import zipfile
import requests
from io import BytesIO
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# URL file ZIP GitHub
zip_url = "https://github.com/C4AnN/Javanese-Script-Augmentation/archive/refs/heads/main.zip"

# Nama folder hasil ekstrak
extract_to = "Javanese_Script"

# Unduh ZIP dari GitHub
print("Mengunduh data dari GitHub...")
response = requests.get(zip_url)
if response.status_code == 200:
    with zipfile.ZipFile(BytesIO(response.content)) as zip_ref:
        zip_ref.extractall()
        print("Ekstraksi selesai.")
else:
    raise Exception("Gagal mengunduh ZIP dari GitHub")

# Rename folder hasil ekstrak (opsional, untuk mempermudah akses)
original_folder = "Javanese-Script-Augmentation-main/Javanese Script"
if not os.path.exists(extract_to):
    os.rename(original_folder, extract_to)

# Inisialisasi ImageDataGenerator
image_generator = ImageDataGenerator(rescale=1./255)
size_w = 64
size_h = size_w

# Buat data generator
train_data = image_generator.flow_from_directory(
    os.path.join(extract_to, "train"),
    target_size=(size_w, size_h),
    batch_size=1,
    class_mode='categorical',
    color_mode='rgb'
)

validation_data = image_generator.flow_from_directory(
    os.path.join(extract_to, "val"),
    target_size=(size_w, size_h),
    batch_size=1,
    class_mode='categorical',
    color_mode='rgb'
)

test_data = image_generator.flow_from_directory(
    os.path.join(extract_to, "test"),
    target_size=(size_w, size_h),
    batch_size=1,
    class_mode='categorical',
    color_mode='rgb'
)


In [ ]:
def switch_dict_key_values(this_dict):
  return dict((v,k) for k,v in this_dict.items())

classes_name = switch_dict_key_values(train_data.class_indices)
print(classes_name)

## **Created CNN Model Architecture**

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image as img_
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [ ]:
model = Sequential()
kernel = 3

model.add(Conv2D(filters=64, kernel_size=kernel, input_shape=(size_w, size_h, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=kernel, activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=256, kernel_size=kernel, activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=512, kernel_size=kernel, activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(20, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
model.summary()

In [ ]:
r = model.fit(
    train_data,
    epochs=5,
    validation_data=validation_data,
)

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(r.history['loss'], label='Loss')
plt.plot(r.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

plt.subplot(2, 2, 2)
plt.plot(r.history['accuracy'], label='Accuracy')
plt.plot(r.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')

In [ ]:
evaluation = model.evaluate(test_data)
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")

In [ ]:
model.save('./model.h5')

In [ ]:
def sort_contours(cnts, method="left-to-right"):
    reverse = False
    i = 0
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
    key=lambda b:b[1][i], reverse=reverse))
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)


def get_letters(img):
    letters = []
    image = cv2.imread(img)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret,thresh1 = cv2.threshold(gray ,127,255,cv2.THRESH_BINARY_INV)
    dilated = cv2.dilate(thresh1, None, iterations=2)

    cnts = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sort_contours(cnts, method="left-to-right")[0]
    # loop over the contours
    for c in cnts:
        count = 1
        pad = 0.1
        if cv2.contourArea(c) > 10:
            (x, y, w, h) = cv2.boundingRect(c)
#             cv2.rectangle(box_img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        roi = image[y:y + h, x:x + w]
        bordersize = int(0.1*x)
        roi = cv2.copyMakeBorder(
            roi,
            top=bordersize,
            bottom=bordersize,
            left=bordersize,
            right=bordersize,
            borderType=cv2.BORDER_CONSTANT,
            value = [255,255,255]
        )
        thresh = cv2.resize(roi, (64, 64))
        img = img_.img_to_array(thresh)
        img = np.expand_dims(img, axis = 0)

        test_gen = ImageDataGenerator(
            rescale = 1./255
        )

        image_gen = test_gen.flow(img)

        ypred = model.predict(image_gen)
        ypred = np.argmax(ypred,axis=1)
        [x] = ypred
        letters.append(str(classes_name[x]))

        cv2.putText(img=roi, text=str(classes_name[x]), org=(0, int(roi.shape[1]*0.1)), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(255, 0, 0))
        plt.figure()
        plt.imshow(roi)

    return letters, image

def get_word(letter):
    word = "".join(letter)
    return word

In [ ]:
import imutils
import matplotlib.pyplot as plt

# Misal 'get_letters' dan 'get_word' sudah didefinisikan sebelumnya

letter, image = get_letters('Javanese_Script/predict_test/dhahara.jpg')
word = get_word(letter)
print('predicted :  ' + word)
plt.figure()
plt.imshow(image)
plt.show()
